In [ ]:
# This is the group project for the MDS5001 Introduction to Python
# Date: 2019.8.12
# Build our own 3-Factor Asset Pricing Model

# Stocks number must be over 30, plus a market index
# Data source: Yahoo Finance

In [ ]:
# import necessary library

import pandas as pd
import requests as rq
import numpy as np
import time
import sys

from sklearn import linear_model
from scipy import stats
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline

# safely disable this new warning with the following assignment.
pd.options.mode.chained_assignment = None

In [ ]:
# Extend the class to get more information
# from https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression

class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """

    def __init__(self, *args, **kwargs):
        super(LinearRegression, self)\
                .__init__(*args, **kwargs)

    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)
        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([
            np.sqrt(np.diagonal(sse[i] * np.linalg.inv(np.dot(X.T, X))))
                                                    for i in range(sse.shape[0])
                    ])

        self.t = self.coef_ / se
        self.p = 2 * (1 - stats.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1]))

        return self

# Data scraping,  Data processing and Feature Engineering

## Data scraping

- data from yahoo finance & daily treasury
- **stocks** use a list of sp500 firms, which is sorted by the market cap of firms
- **market index** use sp500
- **risk free rate** use 1-month treasury bill rate
- use ```request.get()``` and ```pd.read_html()```

### Notice that we can only scrap the data of 100 days at one time, so we need to scrap more than once

- caculate epoch time of 100 days
- scrap from the end date, and minus Epoch Time of 100 days in loop, until start date is bigger than end date

### Save data

- After scraping all the historical data of sp500, use ```to_csv()``` to save them
- Then, we can load data from laptop instead of the Internet ( much faster )

In [ ]:
# Function: change format date into epoch time
def dateToEpoch(date):
    time_tuple = time.strptime(date, '%Y-%m-%d')
    time_epoch = time.mktime(time_tuple)
    time_tostr = str(int(time_epoch))
    
    return time_tostr

In [ ]:
# Function: used to scrap data from yahoo finance
# parameter: 
#     data_list  - a list of string, stocks name or market index
#     start_date - historical data start date
#     end_date   - historical data end date
# return:
#     df_list    - a list of dataframe

def dataScraping(data_list, start_date, end_date):
    
    yahoo_finance_url = 'https://au.finance.yahoo.com/quote/%s/history?period1=%s&period2=%s&interval=1d&filter=history&frequency=1d'
    df_list = []
    count = 0
    one_hundred_epoch = 60 * 60 * 24 * 100      # used to get 100 days of data step by step
    
    for data in data_list:
        print("Scraping", data, "...")
        # try to get web page if end_date > start_date
        first_round = True
        temp_date   = end_date
        while temp_date > start_date:
            yahoo_hist_price_page = rq.get(yahoo_finance_url % (data, start_date, temp_date))
            if yahoo_hist_price_page.status_code == 200:
                if first_round:
                    df_list.append(pd.read_html(yahoo_hist_price_page.text)[0].iloc[:-1]) # exclude last row
                    first_round = False
                else:
                    df_combined = pd.read_html(yahoo_hist_price_page.text)[0].iloc[:-1]
                    df_list[count] = pd.concat([df_list[count], df_combined], axis=0)     # more efficient
                    # df_list[count] = df_list[count].append(pd.read_html(yahoo_hist_price_page.text)[0].iloc[:-1])
            else:
                raise Exception('Some Error Happened. Http Error Code %d' % yahoo_hist_price_page.status_code)

            temp_date = str(int(temp_date) - one_hundred_epoch)

        count += 1
    print("----------------------\nScraping process done!")
    
    return df_list

In [ ]:
# Read sp500 firms name, which is sorted by market cap
result=[]
with open('SP500_Stocks.txt','r') as f:
    for line in f:
        result.append(line.strip('\n').split()[0])

In [ ]:
# Scraping data

security_list   = result                      # Security list
market_index_list = ['%5EGSPC']               # Market index list
start_date = dateToEpoch('2018-07-01')
end_date   = dateToEpoch('2019-06-30')

# data from yahoo finance, stocks and market index
# stocks_df = dataScraping(security_list, start_date, end_date)
# market_index_df = dataScraping(market_index_list, start_date, end_date)

In [ ]:
# data from daily treasury
# scraping the 1-month treasury bill rate 2018 & 2019

years = ['2018', '2019']
treasury_url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=billRatesYear&year=%s'
first_bool = True

for year in years:
    
    treasury_page = rq.get(treasury_url % year)

    if treasury_page.status_code == 200:
        bill_rate_df = pd.read_html(treasury_page.text)[1]
    else:
        raise Exception('Some Error Happened. Http Error Code %d' % treasury_page.status_code)
    
    if first_bool:
        bill_rate_month_df = pd.DataFrame()
        bill_rate_month_df['Date'] = bill_rate_df['Unnamed: 0_level_0']['DATE']
        bill_rate_month_df['Month Rate'] = bill_rate_df['4 WEEKS']['BANK DISCOUNT']
        # convert time to datetime
        bill_rate_month_df['Date'] = pd.to_datetime(bill_rate_month_df['Date'])
    else:
        new_month_df = pd.DataFrame()
        new_month_df['Date'] = bill_rate_df['Unnamed: 0_level_0']['DATE']
        new_month_df['Month Rate'] = bill_rate_df['4 WEEKS']['BANK DISCOUNT']
        # convert time to datetime
        new_month_df['Date'] = pd.to_datetime(new_month_df['Date'])
        
        # concat two dataframe
        bill_rate_month_df = pd.concat([bill_rate_month_df, new_month_df], axis = 0)

    first_bool = False

bill_rate_month_df.head()

In [ ]:
# Saving Data to csv

# bill_rate_month_df.to_csv('bill_rate.csv', index=False)
# market_index_df[0].to_csv('market_index.csv', index=False)
# for i in range(len(stocks_df)):
#    stocks_df[i].to_csv('./data/stocks' + str(i) + '.csv')
# for i in range(len(market_index_df)):
#     market_index_df[i].to_csv('./data/market' + str(i) + '.csv', index=False)

## Data Processing

- load data from csv

### Data cleaning

- drop duplicate rows, drop rows contain UNEXPECTED STRING
- change the value to numerical value
- change date format used ```pd.to_datetime()```

### Data pre-processing

- add stocks name as a column
- merge bill rate dataframe
- reverse the dataframe
- reset the index of dataframe

In [ ]:
# Loading Data from csv saved before

# Load stocks data from 2018/7/1 to 2019/6/30
stocks_df = []
for i in range(len(security_list)):
    stocks_df.append(pd.read_csv('./data/stocks' + str(i) + '.csv').drop(['Unnamed: 0'],axis=1))

# Load market index
market_index_df = []
for i in range(len(market_index_list)):
    market_index_df.append(pd.read_csv('market_index.csv'))
# Load bill rate
bill_rate_month_df = pd.read_csv('bill_rate.csv')
bill_rate_month_df['Date'] = pd.to_datetime(bill_rate_month_df['Date'])

In [ ]:
# Function: used to cleaning data

def dataCleaning(df_list, data_list):
    for i in range(len(df_list)):
        # Drop 'dividend' rows
        if np.issubdtype(df_list[i]['Open'][-1:], np.string_):
            df_list[i] = df_list[i][~df_list[i]['Open'].str.contains('Dividend')]
        
        # change numeric str to numeric values
        for column in df_list[i].columns:
            if column == 'Date':
                continue
            df_list[i][column] = pd.to_numeric(df_list[i][column], errors='coerce')
        # Drop duplicated data
        df_list[i].drop_duplicates(["Date"], keep="first", inplace=True)

        # Add security name as a column
        df_list[i]['Name'] = str(data_list[i])
        # Convert time format
        df_list[i]['Date'] = pd.to_datetime(df_list[i]['Date'])
        # Merge risk-free rate by Date
        df_list[i] = pd.merge(df_list[i], bill_rate_month_df, how='inner', on=['Date'])
        # Reverse dataframe
        df_list[i] = df_list[i][::-1]
        # Reset index
        df_list[i] = df_list[i].reset_index(drop=True)

In [ ]:
# Cleaning Data
dataCleaning(stocks_df, security_list)
dataCleaning(market_index_df, market_index_list)

In [ ]:
# check if there is NAN value
# print(stocks_df[0].isnull().any()) 

## Feature Engineering

In [ ]:
# Feature Engineering

for i in range(len(market_index_df)):
    market_index_df[i]['Pct_change'] = market_index_df[i]['Adj. close**'].pct_change(periods = 1)
    market_index_df[i]['Rm - Rf'] = market_index_df[i]['Pct_change'] - stocks_df[0]['Month Rate'].shift() / 100 / 365

for i in range(len(stocks_df)):
    
    # Ri of stocks
    stocks_df[i]['Pct_change'] = stocks_df[i]['Adj. close**'].pct_change(periods = 1)
    # Ri - Rf of stocks
    stocks_df[i]['Ri - Rf'] = stocks_df[i]['Pct_change'] - stocks_df[0]['Month Rate'].shift() / 100 / 365
    # Log volume
    stocks_df[i]['Log_Volume'] = np.log(stocks_df[i]['Volume'])
    
    # Rmkt - Rf from market_index, used sp500
    stocks_df[i] = pd.merge(stocks_df[i], market_index_df[0][['Date', 'Rm - Rf']], how='inner', on=['Date'])

    
#   Caculate log return for each stock
#   stocks_df[i]['Log_return'] = np.log(stocks_df[i]['Adj. close**']) / np.log(stocks_df[i]['Adj. close**'].shift(-1))

In [ ]:
# Featuring factor of Rs - Rm

mid = len(stocks_df) // 2

big_merge_df   = stocks_df[0][['Date', 'Pct_change']]
small_merge_df = stocks_df[mid][['Date', 'Pct_change']]
for i in range(1, mid):
    big_merge_df = pd.merge(big_merge_df, stocks_df[i][['Date', 'Pct_change']], how='outer', on=['Date'])
for i in range(mid+1, len(stocks_df)):
    small_merge_df = pd.merge(small_merge_df, stocks_df[i][['Date', 'Pct_change']], how='outer', on=['Date'])

In [ ]:
# Caculate the mean of big and small company returns
big_merge_df['mean return']   = big_merge_df.mean(axis=1)
small_merge_df['mean return'] = small_merge_df.mean(axis=1)

new_merge_df = pd.merge(big_merge_df[['Date', 'mean return']], small_merge_df[['Date', 'mean return']], \
                        how='inner', on=['Date'])
new_merge_df['Rs - Rb'] = new_merge_df['mean return_y'] - new_merge_df['mean return_x']

In [ ]:
# merge Rs - Rb in stocks_df
for i in range(len(stocks_df)):
    stocks_df[i] = pd.merge(stocks_df[i], new_merge_df[['Date', 'Rs - Rb']], how='inner', on=['Date'])

In [ ]:
# Do the regression and save information
score_list = []
name_list  = []
alpha_list = []
F1_list = []
F2_list = []
F3_list = []
P_F1_list = []
P_F2_list = []
P_F3_list = []

factor_list = ['Rm - Rf', 'Rs - Rb', 'Log_Volume']

for stock in stocks_df:
    
    # drop rows contain nan
    stock.dropna(axis=0, inplace=True)
    
    if stock.shape[0] <= 0:
        continue
    
    X_train = stock[factor_list]
    Y_train = stock['Ri - Rf']
    
    # data standardised
    scaler = StandardScaler()
    
    for factor in factor_list:
        X_train[factor] = scaler.fit_transform(X_train[factor].values.reshape(-1,1))
    Y_train = scaler.fit_transform(Y_train.values.reshape(-1,1))
    
    # fit linear regression model 
    reg = LinearRegression(normalize=True, n_jobs=4).fit(X_train, Y_train)
    
    # Save information
    score_list.append(reg.score(X_train, Y_train))
    name_list.append(stock['Name'][-1:].values[0])
    alpha_list.append(reg.intercept_[0])
    F1_list.append(reg.coef_[0, 0])
    F2_list.append(reg.coef_[0, 1])
    F3_list.append(reg.coef_[0, 2])
    P_F1_list.append(reg.p[0, 0])
    P_F2_list.append(reg.p[0, 1])
    P_F3_list.append(reg.p[0, 2])

In [ ]:
plt.scatter(X_train['Rm - Rf'], Y_train)
plt.show()

In [ ]:
plt.scatter(X_train['Rs - Rb'], Y_train)
plt.show()

In [ ]:
plt.scatter(X_train['Log_Volume'], Y_train)
plt.show()

In [ ]:
# construct an output dataframe for all the regression models
output_df = pd.DataFrame(columns=['Name', 'R^2', 'alpha', 'F1', 'F2', 'F3', 'P_F1', 'P_F2', 'P_F3'])

output_df['Name']  = name_list
output_df['R^2']   = score_list
output_df['alpha'] = alpha_list
output_df['F1']    = F1_list
output_df['F2']    = F2_list
output_df['F3']    = F3_list
output_df['P_F1']  = P_F1_list
output_df['P_F2']  = P_F2_list
output_df['P_F3']  = P_F3_list

In [ ]:
output_df.describe()

In [ ]:
# analysis in different industry
energy_list = ['XOM','CVX','COP','KMI','SLB','PSX','EOG','OXY','VLO','MPC','OKE','WMB','BHGE','PXD','HES']
tech_list   = ['MSFT','AAPL','GOOG','GOOGL','FB','INTC','CSCO','ORCL','ADBE','CRM','ACN','IBM','TXN','AVGO','NVDA']
health_list = ['JNJ','UNH','MRK','PFE','ABT','MDT','AMGN','TMO','LLY','DHR','ABBV','SYK','GILD','CVS','BMY']

In [ ]:
energy_df = output_df[output_df['Name'].isin(energy_list)]
tech_df   = output_df[output_df['Name'].isin(tech_list)]
health_df = output_df[output_df['Name'].isin(health_list)]

In [ ]:
energy_stock_df = []
for stock in stocks_df:
    if (stock['Name'].isin(energy_list)).any():
        energy_stock_df.append(stock)
tech_stock_df = []
for stock in stocks_df:
    if (stock['Name'].isin(tech_list)).any():
        tech_stock_df.append(stock)

In [ ]:
# daily return change in energy industry
fig = plt.figure(figsize=(20,8))
plt.plot(energy_stock_df[0]['Date'], energy_stock_df[0]['Pct_change'], c='red')
plt.plot(energy_stock_df[1]['Date'], energy_stock_df[1]['Pct_change'], c='blue')
plt.show()

In [ ]:
energy_merge_df = pd.DataFrame()
for stock in energy_stock_df:
    energy_merge_df = pd.concat([energy_merge_df, stock], axis = 0)
tech_merge_df = pd.DataFrame()
for stock in tech_stock_df:
    tech_merge_df = pd.concat([tech_merge_df, stock], axis = 0)

In [ ]:
# average return in energy & technology industry
fig = plt.figure(figsize=(20,8))
grouped_energy_mean = energy_merge_df.groupby('Date')['Pct_change'].mean()
grouped_tech_mean = tech_merge_df.groupby('Date')['Pct_change'].mean()
plt.plot(grouped_energy_mean.index, grouped_energy_mean, c='purple', label='energy')
plt.plot(grouped_tech_mean.index, grouped_tech_mean, c='blue', label='tech')
plt.legend()
plt.show()